In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
import dlt

In [0]:
@dlt.table(
    name = 'customer_stagging_table',
    table_properties = {
        'location': 'abfss://gold@osspstorageaccount.dfs.core.windows.net/stagging_table/customer_stagging_table'
    } 
)

def customer_stagging_table():
    df = spark.readStream.format('delta')\
        .load('abfss://silver@osspstorageaccount.dfs.core.windows.net/customer')
    return df

In [0]:
@dlt.table(
    name = 'date_stagging_table',
    table_properties = {
        'location': 'abfss://gold@osspstorageaccount.dfs.core.windows.net/stagging_table/date_stagging_table'
    } 
)

def date_stagging_table():
    df = spark.readStream.format('delta')\
        .load('abfss://silver@osspstorageaccount.dfs.core.windows.net/date')
    return df

In [0]:
@dlt.table(
    name = 'paymentmethod_stagging_table',
    table_properties = {
        'location': 'abfss://gold@osspstorageaccount.dfs.core.windows.net/stagging_table/paymentmethod_stagging_table'
    } 
)

def paymentmethod_stagging_table():
    df = spark.readStream.format('delta')\
        .load('abfss://silver@osspstorageaccount.dfs.core.windows.net/paymentmethod')
    return df

In [0]:
@dlt.table(
    name = 'product_stagging_table',
    table_properties = {
        'location': 'abfss://gold@osspstorageaccount.dfs.core.windows.net/stagging_table/product_stagging_table'
    } 
)

def product_stagging_table():
    df = spark.readStream.format('delta').load('abfss://silver@osspstorageaccount.dfs.core.windows.net/product')
    return df

In [0]:
@dlt.table(
    name = 'promotion_stagging_table',
    table_properties = {
        'location': 'abfss://gold@osspstorageaccount.dfs.core.windows.net/stagging_table/promotion_stagging_table'
    } 
)
def promotion_stagging_table():
    df = spark.readStream.format('delta').load('abfss://silver@osspstorageaccount.dfs.core.windows.net/promotion')
    return df

In [0]:
@dlt.table(
    name = 'region_stagging_table',
    table_properties = {
        'location': 'abfss://gold@osspstorageaccount.dfs.core.windows.net/stagging_table/region_stagging_table'
    } 
)

def region_stagging_table():
    df = spark.readStream.format('delta').load('abfss://silver@osspstorageaccount.dfs.core.windows.net/region')
    return df

In [0]:
@dlt.table(
    name = 'shippingmethod_stagging_table',
    table_properties = {
        'location': 'abfss://gold@osspstorageaccount.dfs.core.windows.net/stagging_table/shippingmethod_stagging_table'
    } 
)

def shippingmethod_stagging_table():
    df = spark.readStream.format('delta').load('abfss://silver@osspstorageaccount.dfs.core.windows.net/shippingmethod')
    return df

In [0]:
import dlt

In [0]:
@dlt.view(
    name = 'dim_customer_transformation',
)
def dim_customer_transformation():
    df = spark.readStream.table('customer_stagging_table')
    df = df.withColumn('sg_customerkey',(unix_timestamp(col('updated_at'))* 1000000) +col('CustomerKey'))
    return df

In [0]:
dlt.create_streaming_table(
    name = 'dim_streaming_customer',
    table_properties = {
    'location': 'abfss://gold@osspstorageaccount.dfs.core.windows.net/dim_table/dim_customer',
    'delta.enableChangeDataFeed': 'true',
    'delta.autoOptimize.optimizeWrite': 'true',
    'delta.autoOptimize.autoCompact': 'true'
}
    )
dlt.apply_changes(
        target = 'dim_streaming_customer',
        source = 'dim_customer_transformation',
        keys = ['CustomerKey'],
        sequence_by = 'updated_at',
        stored_as_scd_type = 2
    )

In [0]:
@dlt.table(
    name = 'dim_customer'
)
def dim_customer():
    df = spark.read.table('dim_streaming_customer')
    return df

In [0]:
@dlt.view(
    name = 'dim_date_transformation',
)

def dim_date_transformation():
    df = spark.readStream.table('date_stagging_table')
    df = df.withColumn('sg_datekey',(unix_timestamp(col('updated_at')) * 1000000) + col('DateKey'))
    return df

In [0]:
dlt.create_streaming_table(
    name = 'dim_streaming_date',
    table_properties = {
    'location': 'abfss://gold@osspstorageaccount.dfs.core.windows.net/dim_table/dim_date',
    'delta.enableChangeDataFeed': 'true',
    'delta.autoOptimize.optimizeWrite': 'true',
    'delta.autoOptimize.autoCompact':'true'
    }

)
dlt.apply_changes(
        target = 'dim_streaming_date',
        source = 'dim_date_transformation',
        keys = ['DateKey'],
        sequence_by = 'updated_at',
        stored_as_scd_type = 2
    )

In [0]:
@dlt.table(
    name = 'dim_date'
)
def dim_date():
    df = spark.read.table('dim_streaming_date')
    return df

In [0]:
@dlt.view(
    name = 'dim_paymentmethod_transformation',

)
def dim_paymentmethod_transformation():
    df = spark.readStream.table('paymentmethod_stagging_table')
    df = df.withColumn('sg_paymentmethodkey', (unix_timestamp(col('updated_at'))*1000000) + col('PaymentKey'))
    return df



In [0]:
dlt.create_streaming_table(
    name = 'dim_streaming_paymentmethod',
    table_properties = {
    'location': 'abfss://gold@osspstorageaccount.dfs.core.windows.net/dim_table/dim_paymentmethod',
    'delta.enableChangeDataFeed':'true',
    'delta.autoOptimize.optimizeWrite':'true',
    'delta.autoOptimize.autoCompact':'true'
} 
)
dlt.apply_changes(
        target = 'dim_streaming_paymentmethod',
        source = 'dim_paymentmethod_transformation',
        keys = ['PaymentKey'],
        sequence_by = 'updated_at',
        stored_as_scd_type = 2
    )

In [0]:
@dlt.table(
    name = 'dim_paymentmethod'
)
def dim_paymentmethod():
    df = spark.read.table('dim_streaming_paymentmethod')
    return df

In [0]:
@dlt.view(
    name = 'dim_product_transformation',
)
def dim_product_transformation():
    df = spark.readStream.table('product_stagging_table')
    df = df.withColumn('sg_productkey',(unix_timestamp(col('updated_at'))*1000000) + col('ProductKey'))
    return df

In [0]:
dlt.create_streaming_table(
    name = 'dim_streaming_product',
    table_properties = {
        'location': 'abfss://gold@osspstorageaccount.dfs.core.windows.net/dim_table/dim_product',
        'delta.enableChangeDataFeed':'true',
        'delta.autoOptimize.optimizeWrite':'true',
        'delta.autoOptimize.autoCompact':'true'
    }
)
dlt.apply_changes(
        target = 'dim_streaming_product',
        source = 'dim_product_transformation',
        keys = ['ProductKey'],
        sequence_by = 'updated_at',
        stored_as_scd_type = 2
        )

In [0]:
@dlt.table(
    name = 'dim_product'
)
def dim_product():
    df = spark.read.table('dim_streaming_product')
    return df

In [0]:
@dlt.view(
    name = 'dim_promotion_transformation',
)
def dim_promotion_transformation():
    df = spark.readStream.table('promotion_stagging_table')
    df = df.withColumn('sg_promotionkey',(unix_timestamp(col('updated_at'))*1000000) + col('PromotionKey'))
    return df

In [0]:
dlt.create_streaming_table(
    name = 'dim_streaming_promotion',
    table_properties = {
        'location': 'abfss://gold@osspstorageaccount.dfs.core.windows.net/dim_table/dim_promotion',
        'delta.enableChangeDataFeed': 'true',
        'delta.autoOptimize.optimizeWrite': 'true',
        'delta.autoOptimize.autoCompact': 'true'

}
)
dlt.apply_changes(
        target = 'dim_streaming_promotion',
        source = 'dim_promotion_transformation',
        keys = ['PromotionKey'],
        sequence_by = 'updated_at',
        stored_as_scd_type = 2
    )

In [0]:
@dlt.table(
    name = 'dim_promotion'
)
def dim_promotion():
    df = spark.read.table('dim_streaming_promotion')
    return df

In [0]:
@dlt.view(
    name = 'dim_region_transformation',
)
def dim_region_transformation():
    df = spark.readStream.table('region_stagging_table')
    df = df.withColumn('sg_regionkey',(unix_timestamp(col('updated_at'))*1000000) + col('RegionKey'))
    return df

In [0]:
dlt.create_streaming_table(
    name = 'dim_streaming_region',
    table_properties = {
        'location': 'abfss://gold@osspstorageaccount.dfs.core.windows.net/dim_table/dim_region',
        'delta.enableChangeDataFeed': 'true',
        'delta.autoOptimize.optimizeWrite': 'true',
        'delta.autoOptimize.autoCompact': 'true'
}
)
dlt.apply_changes(
        target = 'dim_streaming_region',
        source = 'dim_region_transformation',
        keys = ['RegionKey'],
        sequence_by = 'updated_at',
        stored_as_scd_type = 2
    )

In [0]:
@dlt.table(
    name = 'dim_region'
)
def dim_region():
    df = spark.read.table('dim_streaming_region')
    return df

In [0]:
@dlt.view(
    name = 'dim_shippingmethod_transformation',

)

def dim_shippingmethod_transformation():
    df = spark.readStream.table('shippingmethod_stagging_table')
    df = df.withColumn('sg_shippingmethodkey',(unix_timestamp(col('updated_at'))*1000000) + col('ShippingKey'))
    return df


In [0]:
dlt.create_streaming_table(
    name = 'dim_streaming_shippingmethod',
    table_properties = {
        'location': 'abfss://gold@osspstorageaccount.dfs.core.windows.net/dim_table/dim_shippingmethod',
        'delta.enableChangeDataFeed': 'true',
        'delta.autoOptimize.optimizeWrite': 'true',
        'delta.autoOptimize.autoCompact': 'true'
}

)
dlt.apply_changes(
        target = 'dim_streaming_shippingmethod',
        source = 'dim_shippingmethod_transformation',
        keys = ['ShippingKey'],
        sequence_by = 'updated_at',
        stored_as_scd_type = 2
    )

In [0]:
@dlt.table(
    name = 'dim_shippingmethod'
)
def dim_shippingmethod():
    df = spark.read.table('dim_streaming_shippingmethod')
    return df

In [0]:
@dlt.table(
    name = 'stagging_fact_table'
)
def stagging_fact_table():
    df = spark.readStream.format('delta').load('abfss://silver@osspstorageaccount.dfs.core.windows.net/factsales')
    return df

In [0]:
@dlt.table(
    name = 'fact_sales_transformation'
)
def fact_sales_transformation():
    df = spark.readStream.table('stagging_fact_table')
    dim_customer = dlt.read('dim_customer')
    dim_date = dlt.read('dim_date')
    dim_payment = dlt.read('dim_paymentmethod')
    dim_product = dlt.read('dim_product')
    dim_promotion = dlt.read('dim_promotion')
    dim_region = dlt.read('dim_region')
    dim_shipping = dlt.read('dim_shippingmethod')

    df = (
        df
        .join(dim_customer, df.CustomerKey == dim_customer.CustomerKey, 'left')
        .join(dim_date, df.DateKey == dim_date.DateKey, 'left')
        .join(dim_payment, df.PaymentKey == dim_payment.PaymentKey, 'left')
        .join(dim_product, df.ProductKey == dim_product.ProductKey, 'left')
        .join(dim_promotion, df.PromotionKey == dim_promotion.PromotionKey, 'left')
        .join(dim_region, df.RegionKey == dim_region.RegionKey, 'left')
        .join(dim_shipping, df.ShippingKey == dim_shipping.ShippingKey, 'left')
    )
    df = df.select('SalesKey','sg_customerkey','sg_datekey','sg_paymentmethodkey','sg_productkey','sg_promotionkey','sg_regionkey','sg_shippingmethodkey','Quantity','UnitPrice','TotalRevenue')
    df.withColumnRenamed('SalesKey','sg_saleskey').withColumnRenamed('Quantity','quantity').withColumnRenamed('UnitPrice','unitprice').withColumnRenamed('TotalRevenue','totalrevenue')
    return df


In [0]:
@dlt.table(
    name = 'fact_sales',
    table_properties ={
        'location': 'abfss://gold@osspstorageaccount.dfs.core.windows.net/fact_table/fact_sales'
    }
)
def fact_sales():
    df = spark.read.table('fact_sales_transformation')
    return df